# Dockless Scooter Data

In [7]:
import matplotlib, shapely, geopandas
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time, datetime
from sklearn.linear_model import LinearRegression

In [8]:
tract_to_name = {
    '000204': "Triangle",
    '000500': "North Campus",
    '000700': "South Campus",
    '000401': "East Campus",
    '000604': "Lower West Campus",
    '000603': "Upper West Campus",
    '000601': "Campus"
}

tract_to_index = {
    '000204': 0,
    '000500': 1,
    '000700': 2,
    '000401': 3,
    '000604': 4,
    '000603': 5,
    '000601': 6,
}

In [9]:
scooter_data = pd.read_csv('~/work/data/Dockless_Vehicle_Trips.csv')
austin_data = austin = geopandas.GeoDataFrame.from_file('/home/jovyan/work/data/census_tracts_2010_msa/census_tracts_2010_msa.shp',)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dropping the first 6 digits of the tract start and end ids and converting to a numerical type.

In [10]:
scooter_data = scooter_data.dropna(subset=['Census Tract Start'])
                                   
scooter_data.sample(5)

,ID,Device ID,Vehicle Type,Trip Duration,Trip Distance,Start Time,End Time,Modified Date,Month,Hour,Day of Week,Council District (Start),Council District (End),Year,Census Tract Start,Census Tract End
606126,e649a626-e182-40e3-8283-d02dff1f0086,b8d5e14a-b19d-49a7-acad-2f4b6ef2dd8d,scooter,1048.0,675.0,09/09/2018 12:45:00 PM,09/09/2018 01:00:00 PM,04/17/2019 02:30:05 AM,9.0,12.0,0.0,9.0,9.0,2018.0,48453001100,48453001100
4161218,1c295a3f-9766-4f4e-bdb4-a63a6ade1124,260dfc04-ba0c-42c4-9fc8-5797410bf889,scooter,532.0,2821.0,03/07/2019 01:00:00 PM,03/07/2019 01:00:00 PM,04/17/2019 02:35:02 AM,3.0,13.0,4.0,9.0,9.0,2019.0,48453000601,48453000302
3342908,89e7502a-3a58-4b8f-8596-f5a7dbc5c299,7af67c4e-fc4b-4d5a-b230-575dc343e28a,scooter,184.0,543.0,12/08/2018 01:30:00 PM,12/08/2018 01:45:00 PM,04/17/2019 03:25:44 PM,12.0,13.0,6.0,9.0,9.0,2018.0,48453000601,48453000603
415352,cef68c41-f50c-41ee-9b87-979b790ca3f3,78a85ecc-adca-4f76-80bd-ba7fb892bbf2,scooter,1912.0,2937.0,07/31/2018 10:45:00 AM,07/31/2018 11:15:00 AM,04/17/2019 01:19:05 AM,7.0,10.0,2.0,9.0,9.0,2018.0,48453001100,48453001100
3491558,e3d871a0-5e34-4446-b5be-eafbb81b9a3e,f24a1ab0-d070-4acc-8157-35e79527e003,scooter,203.0,0.0,02/02/2019 12:00:00 AM,02/02/2019 12:00:00 AM,04/17/2019 02:42:24 AM,2.0,0.0,6.0,2.0,2.0,2019.0,48453002402,48453002402


In [11]:
scooter_data = scooter_data[scooter_data['Census Tract Start'] != 'OUT_OF_BOUNDS']
scooter_data = scooter_data[scooter_data['Census Tract End'] != 'OUT_OF_BOUNDS']
scooter_data['Census Tract Start'] = pd.to_numeric(scooter_data['Census Tract Start'])
scooter_data['Census Tract End'] = pd.to_numeric(scooter_data['Census Tract End'])

In [ ]:
def tract_to_trunk(tract):
    return "%06d" % (int(tract) % 1000000)

scooter_data['tract_start'] = scooter_data['Census Tract Start'].apply(tract_to_trunk)
scooter_data['tract_end'] = scooter_data['Census Tract End'].apply(tract_to_trunk)

In [ ]:
oncampus  = ['000601']
offcampus = ['000204', '000500', '000700', '000401', '000604', '000603']
scooter_data_starting = scooter_data[scooter_data['tract_start'].isin(oncampus + offcampus)]
scooter_data_ending   = scooter_data[scooter_data['tract_end'].isin(oncampus + offcampus)]

In [ ]:
scooter_data_raw = pd.read_csv('~/work/data/Dockless_Vehicle_Trips.csv')
scooter_data_raw.shape

In [ ]:
100 * (scooter_data_starting.shape[0] / scooter_data_raw.shape[0])

In [ ]:
start_freqs = scooter_data_starting.groupby(['tract_start'])['tract_start'].agg('count').to_frame('count').reset_index()
start_data = geopandas.GeoDataFrame(pd.merge(start_freqs, austin_data, how='inner', left_on=['tract_start'], right_on=['TRACTCE10']))
start_data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
start_data.plot(column='count', ax=ax, legend=True)

In [ ]:
end_freqs = scooter_data_ending.groupby(['tract_end'])['tract_end'].agg('count').to_frame('count').reset_index()
end_data = geopandas.GeoDataFrame(pd.merge(end_freqs, austin_data, how='inner', left_on=['tract_end'], right_on=['TRACTCE10']))
end_data.head()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
end_data.plot(column='count', ax=ax, legend=True)

In [ ]:
print(scooter_data_starting.shape, scooter_data_ending.shape)
scooter_data_starting.head()
merged_data =scooter_data_starting[scooter_data_starting['ID'].isin(scooter_data_ending['ID'])]
merged_data.shape

In [ ]:
merged_start_freqs = merged_data.groupby(['tract_start'])['tract_start'].agg('count').to_frame('count').reset_index()
merged_end_freqs = merged_data.groupby(['tract_end'])['tract_end'].agg('count').to_frame('count').reset_index()

merged_start_data = geopandas.GeoDataFrame(pd.merge(merged_start_freqs, austin_data, how='inner', left_on=['tract_start'], right_on=['TRACTCE10']))
merged_end_data = geopandas.GeoDataFrame(pd.merge(merged_end_freqs, austin_data, how='inner', left_on=['tract_end'], right_on=['TRACTCE10']))

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
merged_start_data.plot(column='count', ax=ax, legend=True, cmap='Pastel2')

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
merged_end_data.plot(column='count', ax=ax, legend=True, cmap='Pastel2')

In [ ]:
total = 0
d = []
d_norm = []
labels = []
for loc in offcampus:
    freq = merged_start_freqs[merged_start_freqs["tract_start"] == loc]["count"].iloc[0]
    total += freq
    labels.append(tract_to_name[str(loc)])
    d.append(freq)

for loc in offcampus:
    freq = merged_start_freqs[merged_start_freqs["tract_start"] == loc]["count"].iloc[0]
    d_norm.append(freq / total)

plt.bar(labels, d, align='center', color='#cc5500', alpha=0.7)
plt.ylabel('Start Frequency')
plt.xticks(rotation=-90)
plt.title('Scooters Starting Off-Campus')

plt.show()

In [ ]:
# randomly sample a location :-)
freqqq = [0]*len(offcampus)
for _ in range(1000):
    freqqq[tract_to_index[np.random.choice(offcampus, p=d_norm)]] += 1

plt.bar(labels, freqqq, align='center', color='#cc5500', alpha=0.7)
plt.ylabel('Start Frequency')
plt.xticks(rotation=-90)
plt.title('Scooters Starting Off-Campus')

plt.show()

In [ ]:
fig, ax = plt.subplots(7, 1,figsize=(15, 10))
for i in range(7): # 0 = sunday, 6 = saturday
    time_freq = merged_data[merged_data['Day of Week'] == i].groupby(['Hour'])['Hour'].agg('count').to_frame('count').reset_index()
    ax[i].plot(time_freq['Hour'], time_freq['count'])
    print(time_freq.nlargest(3, "count"))

In [ ]:
merged_data['Start Time'] = pd.to_datetime(merged_data['Start Time'], format="%m/%d/%Y %I:%M:%S %p")
merged_data['End Time']   = pd.to_datetime(merged_data['End Time'],   format="%m/%d/%Y %I:%M:%S %p")

In [ ]:
merged_data['Start Time'].sample()

In [ ]:
def to_decimal(strtime):
    hour, minute = map(int, strtime.strftime("%H:%M").split(':'))
    return hour + (minute / 60)

merged_data['start_hour_decimal'] = merged_data['Start Time'].apply(to_decimal)
merged_data['end_hour_decimal'] = merged_data['End Time'].apply(to_decimal)

In [ ]:
merged_data.head()

In [ ]:
fig, ax = plt.subplots(7, 1,figsize=(15, 10))
for i in range(7): # 0 = sunday, 6 = saturday
    start_time_freq = merged_data[merged_data['Day of Week'] == i].groupby(['start_hour_decimal'])['start_hour_decimal'].agg('count').to_frame('count').reset_index()
    end_time_freq   = merged_data[merged_data['Day of Week'] == i].groupby(['end_hour_decimal'])['end_hour_decimal'].agg('count').to_frame('count').reset_index()
    ax[i].plot(start_time_freq['start_hour_decimal'], start_time_freq['count'], c="blue")
    ax[i].plot(end_time_freq['end_hour_decimal'], end_time_freq['count'], c="red")
#     print(time_freq.nlargest(3, "count"))

In [ ]:
def get_start_time_distribution(class_time): # P(start_time)
    first_time      = class_time - 0.50
    second_time     = class_time - 0.25
    
    start_time_freq = merged_data[merged_data['Day of Week'] == 3].groupby(['start_hour_decimal'])['start_hour_decimal'].agg('count').to_frame('count').reset_index()
    first  = int(start_time_freq[start_time_freq['start_hour_decimal'] == first_time]['count'].iloc[0])
    second = int(start_time_freq[start_time_freq['start_hour_decimal'] == second_time]['count'].iloc[0])
    third  = int(start_time_freq[start_time_freq['start_hour_decimal'] == class_time]['count'].iloc[0])
    
    first_ln, second_ln, third_ln = np.log(first), np.log(second), np.log(third)
    first_piecewise  = LinearRegression().fit(np.array([0,  15]).reshape((-1, 1)), [first,  second])
    second_piecewise = LinearRegression().fit(np.array([15, 30]).reshape((-1, 1)), [second_ln, third_ln])
    
    distribution = {}
    total = 0.0
    for minute in range(0, 30):
        if minute < 15:
            total += first_piecewise.predict(np.array([minute]).reshape(1, -1))[0]
        else:
            total += np.exp(second_piecewise.predict(np.array([minute]).reshape(1, -1))[0])
            
    for minute in range(0, 30):
        t = first_time + (minute / 60)
        if minute < 15:
            distribution[t] = first_piecewise.predict(np.array([minute]).reshape(1, -1))[0]
        else:
            distribution[t] = np.exp(second_piecewise.predict(np.array([minute]).reshape(1, -1))[0])

        distribution[t] /= total
        
    squished_dist = {first_time: 0.0, second_time: 0.0}
    for minute in range(0, 30):
        t = first_time + (minute / 60)
        if minute < 15:
            squished_dist[first_time] += distribution[t]
        else:
            squished_dist[second_time] += distribution[t]

    return squished_dist

distribution = get_start_time_distribution(8.0)
print(distribution)

distribution = get_start_time_distribution(9.0)
print(distribution)

In [ ]:
def get_distance_distribution(start_time): # P(distance | start_time)
    distances_given_start = merged_data[merged_data["start_hour_decimal"] == start_time]
    distances_given_start = distances_given_start[(distances_given_start['Trip Distance'] > 50) & (distances_given_start['Trip Distance'] < 2000)].sort_values(by=['Trip Distance'])
    distances_freq = distances_given_start.groupby(['Trip Distance'])['Trip Distance'].agg('count').to_frame('count').reset_index()
#     return distances_freq
    
    # avg pool/smooth data
    distances_freq_pooled = {}
    total = 0.0
    for mid in range(55, 1906, 10):
        val = distances_freq[distances_freq['count'].isin(list(range(mid, mid + 10)))].mean()
        distances_freq_pooled[mid] = val
        total += val
        
    for mid in range(55, 1906, 10):
        distances_freq_pooled[mid] /= total
        
#     return distances_freq_pooled
    x, y = zip(*sorted(distances_freq_pooled.items()))

    plt.plot(x, y)
    plt.show()

In [ ]:
get_distance_distribution(9.0)

In [ ]:
    distances_given_start = merged_data[merged_data["start_hour_decimal"] == 9.5]
    distances_given_start = distances_given_start[(distances_given_start['Trip Distance'] > 50) & (distances_given_start['Trip Distance'] < 2000)].sort_values(by=['Trip Distance'])
    distances_freq = distances_given_start.groupby(['Trip Distance'])['Trip Distance'].agg('count').to_frame('count').reset_index()